# How to generate text: using different decoding methods for language generation with Transformers

## Import

In [4]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# torch_device = "cuda" if torch.cuda.is_available() else "cpu"

# model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)
# model.save_pretrained("./model/gpt2/")

# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# tokenizer.save_pretrained("./model/gpt2/")

# add the EOS token as PAD token to avoid warnings

/user_data/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/user_data/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2024-01-31 12:10:32.436720: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 12:10:32.453398: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 12:10:32.453420: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 12:10:32.453433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 12:10:32.457315: I tensorflow/core/platform/cpu_feature_g

('./model/gpt2/tokenizer_config.json',
 './model/gpt2/special_tokens_map.json',
 './model/gpt2/vocab.json',
 './model/gpt2/merges.txt',
 './model/gpt2/added_tokens.json',
 './model/gpt2/tokenizer.json')

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained("./model/gpt2")
tokenizer = AutoTokenizer.from_pretrained("./model/gpt2")

/user_data/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/user_data/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/user_data/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2024-01-31 12:10:57.451424: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 12:10:57.469466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 12:10:57.469490: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 12:10:57.469503: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 12:10:57.473096: I tensorflow/core/platform/cpu_feature_g

In [2]:
# encode context the generation is conditioned on
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=False))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure


## 整理

In [ ]:
set_seed(42)

# set num_return_sequences > 1
outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    
    # beam-based generation modes
    num_beams=5,            # 1 for default (no beam search)
    no_repeat_ngram_size=2, # setting the probability of next words that could create an already seen n-gram to 0
    num_return_sequences=5, # the number of highest scoring beams that should be returned. (<= num_beams)
    early_stopping=True,    # generation is finished when all beam hypotheses reached the EOS token (End Of Sequence)
    
    # sampling
    do_sample=True,     # sample accroding to the score of each word
    top_p=0.92,         # 1 for default
    top_k=0,            # 50 for default. 0 for deactivate.
    temperature=0.6,    # 1 for default
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, output in enumerate(outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(output, skip_special_tokens=True)))

## beam search

In [8]:
# # encode context the generation is conditioned on
# model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

# # generate 40 new tokens
# greedy_output = model.generate(**model_inputs, max_new_tokens=100)

# print("Output:\n" + 100 * '-')
# print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my


In [12]:
# activate beam search and early_stopping
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    # no_repeat_ngram_size=2, # setting the probability of next words that could create an already seen n-gram to 0
    early_stopping=True     # generation is finished when all beam hypotheses reached the EOS token (End Of Sequence)
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure


In [17]:
# activate beam search and early_stopping
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2, # setting the probability of next words that could create an already seen n-gram to 0
    early_stopping=True     # generation is finished when all beam hypotheses reached the EOS token (End Of Sequence)
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to


In [18]:
# activate beam search and early_stopping
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2, # setting the probability of next words that could create an already seen n-gram to 0
    # early_stopping=True     # generation is finished when all beam hypotheses reached the EOS token (End Of Sequence)
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to


In [20]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2, # setting the probability of next words that could create an already seen n-gram to 0
    num_return_sequences=5, # the number of highest scoring beams that should be returned. (<= num_beams)
    early_stopping=True     # generation is finished when all beam hypotheses reached the EOS token (End Of Sequence)
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to


1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to


2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea to


3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time to take a


4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a g

## do sample

In [31]:
# set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
set_seed(42)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0             # 50 for default. 0 for deactivate.
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but what I love about being a dog cat person is being a pet being with people who can treat you. I feel happy to be such a pet person and get to meet so many people. I


In [33]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0,
    temperature=0.6,    # 1 for default
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but I also love the fact that my cat is not a dog. She is a good, loving dog. I do not like to be held back by other dogs but I think that I have to


In [36]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.92,
    top_k=0,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


/user_data/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but what I love about being a dog cat person is being a pet being with people who can treat you. I feel happy to be such a pet person and get to meet so many people. I


# Guiding Text Generation with Constrained Beam Search in 🤗 Transformers

## Import

In [3]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("t5-base")
# tokenizer.save_pretrained("./model/t5/")

# model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
# model.save_pretrained("./model/t5/")

/user_data/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
model.safetensors: 100%|██████████| 892M/892M [01:18<00:00, 11.3MB/s] 


## EX1: Constrained Beam Search

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("./model/t5/")
model = AutoModelForSeq2SeqLM.from_pretrained("./model/t5/")

In [15]:
encoder_input_str = "translate English to German: How old are you?"
# informal : "Wie alt bist du?"
#   formal : "Wie alt sind Sie?"

In [4]:
# Traditional Beam Search

input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids

outputs = model.generate(
    input_ids,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
Wie alt bist du?


In [7]:
# Constrained Beam Search

force_words = ["Sie"]

input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids
force_words_ids = tokenizer(force_words, add_special_tokens=False).input_ids

outputs = model.generate(
    input_ids,
    force_words_ids=force_words_ids,
    num_beams=5,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
Wie alt sind Sie?


In [17]:
from transformers import PhrasalConstraint

constraints = [
    PhrasalConstraint(
        tokenizer("Sie", add_special_tokens=False).input_ids
    )
]

input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids


outputs = model.generate(
    input_ids,
    constraints=constraints,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)


print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
Wie alt sind Sie?


## EX2: Disjunctively Constrained Beam Search

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("./model/gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./model/gpt2")

/user_data/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/user_data/.local/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/user_data/.local/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2024-02-02 07:01:22.598662: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-02 07:01:23.040153: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 07:01:23.040245: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 07:01:23.040294: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 07:01:23.233393: I tensorflow/core/platform/cpu_feature_g

In [5]:
force_word = "scaredis"
force_flexible = ["scream", "screams", "screaming", "screamed"]

In [10]:
tokenizer(["scaredis"], add_prefix_space=True, add_special_tokens=False).input_ids

[[12008, 271]]

In [9]:
tokenizer(["scared is"], add_prefix_space=True, add_special_tokens=False).input_ids

[[12008, 318]]

In [11]:
tokenizer(["scared  is"], add_prefix_space=True, add_special_tokens=False).input_ids

[[12008, 220, 318]]

In [13]:
tokenizer(["scared   is"], add_prefix_space=True, add_special_tokens=False).input_ids

[[12008, 220, 220, 318]]

In [2]:
force_word = "scared is"
force_flexible = ["scream", "screams", "screaming", "screamed"]

force_words_ids = [
    tokenizer([force_word], add_prefix_space=True, add_special_tokens=False).input_ids,
    tokenizer(force_flexible, add_prefix_space=True, add_special_tokens=False).input_ids,
]

# 有兩個起始字串，就會生出兩句話，所以最下面 outputs 的 index 可以到 1
starting_text = ["The soldiers", "The child"]
input_ids = tokenizer(starting_text, return_tensors="pt").input_ids

outputs = model.generate(
    input_ids,
    
    pad_token_id=tokenizer.eos_token_id,
    # For warning:
    # The attention mask and the pad token id were not set.
    # As a consequence, you may observe unexpected behavior.
    # Please pass your input's `attention_mask` to obtain reliable results.
    
    max_new_tokens = 50,
    # For warning:
    # Using the model-agnostic default `max_length` (=20) to control the generation length.
    # We recommend setting `max_new_tokens` to control the maximum length of the generation.
    
    force_words_ids=force_words_ids,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)

print(len(outputs))

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print(100 * "*")
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

2
Output:
----------------------------------------------------------------------------------------------------
The soldiers, who had been ordered to leave by the commander of their own country's military intelligence agency (MIA), were told that they could not return home. The men then left for a safe place in an undisclosed location and went back into screaming scared is
****************************************************************************************************
The child, who was born with a congenital heart defect and is now in the care of an obstetrician or gynecologist.
 (Photo: Courtesy photo) Story Highlights The baby's mother died at age 3 after being diagnosed by scared is screams


# Param: Constraints

huggingface - Transformers - API - MAIN CLASSES - Text Generation

https://huggingface.co/docs/transformers/v4.33.3/en/main_classes/text_generation

force_words_ids 跟 constraints，這兩個參數可以同時用。

## PhrasalConstraint

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("./model/t5/")
model = AutoModelForSeq2SeqLM.from_pretrained("./model/t5/")

encoder_input_str = "translate English to German: How old are you?"

/user_data/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-26 09:05:28.314972: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 09:05:28.333126: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 09:05:28.333148: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 09:05:28.333160: E t

In [2]:
from transformers import PhrasalConstraint

constraints = [
    PhrasalConstraint(
        tokenizer("Sie is", add_special_tokens=False).input_ids
    )
]

In [3]:
display(constraints)
for constraint in constraints:
    print(constraint.token_ids)

[292, 19]


In [4]:
input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids


outputs = model.generate(
    input_ids,
    
    # force_words_ids=tokenizer(["456789123"], add_special_tokens=False).input_ids,
    
    constraints=constraints,    # List[Constraint]
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)


print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 3479

advance = 292
advance = 3479

advance = 292
advance = 3479

advance = 3479

advance = 292
advance = 3479

advance = 3479


/user_data/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



advance = 3479

advance = 292
advance = 3479

advance = 3479

advance = 3479

advance = 292
advance = 3479

advance = 3479

advance = 292
advance = 3479

advance = 3479

advance = 292
advance = 3479

advance = 3479

advance = 292
advance = 3479

advance = 3479

advance = 3479

advance = 3479
advance = 292

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479

advance = 3479
Output:
----------------------------------------------------------------------------------------------------
Wie alt bist du? Sie isst ich schon seit 15 Jahren 456789123


In [5]:
display(constraints)
for constraint in constraints:
    print(constraint.token_ids)

[292, 19]
[3479, 3708, 3914, 14574]


In [7]:
qqq = [3479, 3708, 3914, 14574]

for id in qqq:
    print(tokenizer.decode([id], skip_special_tokens=True))

45
67
89
123


## TemplateConstraints (not implement)
template: 範本

In [20]:
starting_text = "The woman"
template = ["the", "", "School of", "", "in"]

# possible_outputs == [
#    "The woman attended the Ross School of Business in Michigan.",
#    "The woman was the administrator for the Harvard School of Business in MA."
# ]


In [21]:
starting_text = "The woman"
template = ["the", "", "", "University", "", "in"]

# possible_outputs == [
#    "The woman attended the Carnegie Mellon University in Pittsburgh.",
# ]
# impossible_outputs == [
#   "The woman attended the Harvard University in MA."
# ]


## Try

### DisjunctiveConstraint

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("./model/gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./model/gpt2")

/user_data/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-26 09:04:32.948500: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 09:04:32.966500: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 09:04:32.966521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 09:04:32.966534: E t

In [2]:
from transformers import DisjunctiveConstraint

force_word = "scared is"
force_flexible = ["scream", "screams", "screaming", "screamed"]

constraints = [
    DisjunctiveConstraint(
        tokenizer([force_word], add_prefix_space=True, add_special_tokens=False).input_ids
    ),
    DisjunctiveConstraint(
        tokenizer(force_flexible, add_prefix_space=True, add_special_tokens=False).input_ids
    ),
    
]

In [3]:
display(constraints)
for constraint in constraints:
    print(constraint.token_ids)

[[12008, 318]]
[[8196], [26557], [14788], [25421]]


In [4]:
# 有兩個起始字串，就會生出兩句話，所以最下面 outputs 的 index 可以到 1
starting_text = ["The soldiers", "The child"]
input_ids = tokenizer(starting_text, return_tensors="pt").input_ids

outputs = model.generate(
    input_ids,
    
    pad_token_id=tokenizer.eos_token_id,
    # For warning:
    # The attention mask and the pad token id were not set.
    # As a consequence, you may observe unexpected behavior.
    # Please pass your input's `attention_mask` to obtain reliable results.
    
    max_new_tokens = 50,
    # For warning:
    # Using the model-agnostic default `max_length` (=20) to control the generation length.
    # We recommend setting `max_new_tokens` to control the maximum length of the generation.
    
    constraints=constraints,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
    
    # do_sample = True
    # # ValueError:
    # # `num_beams` has to be an integer strictly greater than 1, but is 1.
    # # For `num_beams` == 1, one should make use of `greedy_search` instead.
)

print(len(outputs))

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print(100 * "*")
print(tokenizer.decode(outputs[1], skip_special_tokens=True))


advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8196, 26557, 14788, 25421]

advance = [12008]
advance = [8

In [5]:
display(constraints)
for constraint in constraints:
    print(constraint.token_ids)

[[12008, 318]]
[[8196], [26557], [14788], [25421]]


### DocConstraints

In [ ]:
from .DocConstraint import DocConstraint

### TemplateConstraints

In [8]:
from transformers import TemplateConstraint

force_word = "scared"
force_flexible = ["scream", "screams", "screaming", "screamed"]

constraints = [
    TemplateConstraint(
        tokenizer([force_word], add_prefix_space=True, add_special_tokens=False).input_ids
    )
    
]

# 有兩個起始字串，就會生出兩句話，所以最下面 outputs 的 index 可以到 1
starting_text = ["The soldiers", "The child"]
input_ids = tokenizer(starting_text, return_tensors="pt").input_ids

outputs = model.generate(
    input_ids,
    
    pad_token_id=tokenizer.eos_token_id,
    # For warning:
    # The attention mask and the pad token id were not set.
    # As a consequence, you may observe unexpected behavior.
    # Please pass your input's `attention_mask` to obtain reliable results.
    
    max_new_tokens = 50,
    # For warning:
    # Using the model-agnostic default `max_length` (=20) to control the generation length.
    # We recommend setting `max_new_tokens` to control the maximum length of the generation.
    constraints=constraints,
    num_beams=10,
    num_return_sequences=1,
    no_repeat_ngram_size=1,
    remove_invalid_values=True,
)

print(len(outputs))

print("Output:\n" + 100 * '-')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print(100 * "*")
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

ImportError: cannot import name 'TemplateConstraint' from 'transformers' (/user_data/.local/lib/python3.10/site-packages/transformers/__init__.py)

In [4]:
any([0,0,0,0])

False

In [5]:
all([1,2,3,0])

False